<a href="https://colab.research.google.com/github/donlap/stat424/blob/main/3_linear_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import linprog

In [ ]:
result = linprog(c=[-300, -500],
                 A_ub=[[3, 4],
                       [0, 1],
                       [1, 1]],
                 b_ub=[1800, 350, 1000],
                 bounds=[(0, None), (0, None)])

In [ ]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -215000.0
              x: [ 1.333e+02  3.500e+02]
            nit: 0
          lower:  residual: [ 1.333e+02  3.500e+02]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  5.167e+02]
                 marginals: [-1.000e+02 -1.000e+02 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [ ]:
np.random.seed(42)

n, m = 50, 5
X = np.random.rand(n, m)
beta = np.array([2, 4, 6, 8, 10])
error = np.random.rand(n)

y = X@beta + error

In [ ]:
result = linprog(
    c=np.hstack([np.zeros(m), np.ones(n)]),
    A_ub=np.vstack([np.hstack([-X, -np.eye(n)]), np.hstack([X, -np.eye(n)])]),
    b_ub=np.hstack([-y, y]),
    bounds=[(0, None)] * (n + m)
    )

In [ ]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 10.228882727830563
              x: [ 2.542e+00  4.108e+00 ...  0.000e+00  8.173e-01]
            nit: 61
          lower:  residual: [ 2.542e+00  4.108e+00 ...  0.000e+00
                              8.173e-01]
                 marginals: [ 0.000e+00  0.000e+00 ...  2.763e-01
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 2.096e-01  0.000e+00 ...  5.329e-15
                              0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00 ... -0.000e+00
                             -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip

In [ ]:
beta

array([ 2,  4,  6,  8, 10])

In [ ]:
result.x[:m]

array([ 2.54158863,  4.10754577,  5.95383067,  8.13035847, 10.31980951])

In [ ]:
result = linprog(c = np.ones(n))

array([-15.58780428, -18.01612396, -13.28356347, -11.41709014,
       -11.19563646, -11.21507562, -20.47105516, -13.9927014 ,
       -12.86353944, -12.01170028, -24.42932002,  -7.97879112,
       -12.63863149, -17.44587512, -16.84212291,  -9.75454867,
       -10.52009327, -19.91647146, -20.21625846,  -7.21962775,
       -18.44822733, -10.0837208 , -20.46864689, -19.5241648 ,
       -10.77837578, -14.95526451, -15.37075904, -20.21650368,
       -11.87291311,  -9.52219269, -18.30628053, -17.26362942,
       -12.63064598, -10.40194882, -13.33601264, -14.33788751,
       -16.74791762, -15.11843393, -17.91922773, -24.47118354,
       -15.87361979,  -6.09121226, -17.51536318, -20.30453334,
        -9.87131942, -20.70860982, -13.72415927,  -9.86459567,
       -14.92454406, -25.69370447,  15.58780428,  18.01612396,
        13.28356347,  11.41709014,  11.19563646,  11.21507562,
        20.47105516,  13.9927014 ,  12.86353944,  12.01170028,
        24.42932002,   7.97879112,  12.63863149,  17.44